# Feature Engineering and Modelling

---

1. Import packages
2. Load data
3. Modelling

---

## 1. Import packages

In [11]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt

# Shows plots in jupyter notebook
%matplotlib inline

# Set plot style
sns.set(color_codes=True)

---
## 2. Load data

In [13]:
df = pd.read_csv('./data_for_predictions.csv')
df.drop(columns=["Unnamed: 0"], inplace=True)
df.head()

id  cons_12m  cons_gas_12m  cons_last_month  \
0  24011ae4ebbe3035111d65fa7c15bc57  0.000000      4.739944         0.000000   
1  d29c2c54acc38ff3c0614d0a653813dd  3.668479      0.000000         0.000000   
2  764c75f661154dac3a6c254cd082ea7d  2.736397      0.000000         0.000000   
3  bba03439a292a1e166f80264c16191cb  3.200029      0.000000         0.000000   
4  149d57cf92fc41cf94415803a877cb4b  3.646011      0.000000         2.721811   

   forecast_cons_12m  forecast_discount_energy  forecast_meter_rent_12m  \
0           0.000000                       0.0                 0.444045   
1           2.280920                       0.0                 1.237292   
2           1.689841                       0.0                 1.599009   
3           2.382089                       0.0                 1.318689   
4           2.650065                       0.0                 2.122969   

   forecast_price_energy_off_peak  forecast_price_energy_peak  \
0                        0.114481                    0.098142   
1                        0.145711                    0.000000   
2                        0.165794                    0.087899   
3                        0.146694                    0.000000   
4                        0.116900                    0.100015   

   forecast_price_pow_off_peak  ...  months_modif_prod  months_renewal  \
0                    40.606701  ...                  2               6   
1                    44.311378  ...                 76               4   
2                    44.311378  ...                 68               8   
3                    44.311378  ...                 69               9   
4                    40.606701  ...                 71               9   

   channel_MISSING  channel_ewpakwlliwisiwduibdlfmalxowmwpci  \
0                0                                         0   
1                1                                         0   
2                0                                         0   
3                0                                         0   
4                1                                         0   

   channel_foosdfpfkusacimwkcsosbicdxkicaua  \
0                                         1   
1                                         0   
2                                         1   
3                                         0   
4                                         0   

   channel_lmkebamcaaclubfxadlmueccxoimlema  \
0                                         0   
1                                         0   
2                                         0   
3                                         1   
4                                         0   

   channel_usilxuppasemubllopkaafesmlibmsdf  \
0                                         0   
1                                         0   
2                                         0   
3                                         0   
4                                         0   

   origin_up_kamkkxfxxuwbdslkwifmmcsiusiuosws  \
0                                           0   
1                                           1   
2                                           1   
3                                           1   
4                                           1   

   origin_up_ldkssxwpmemidmecebumciepifcamkci  \
0                                           0   
1                                           0   
2                                           0   
3                                           0   
4                                           0   

   origin_up_lxidpiddsbxsbosboudacockeimpuepw  
0                                           1  
1                                           0  
2                                           0  
3                                           0  
4                                           0  

[5 rows x 63 columns]

---

## 3. Modelling

We now have a dataset containing features that we have engineered and we are ready to start training a predictive model. Remember, we only need to focus on training a `Random Forest` classifier.

In [14]:
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

### Data sampling

The first thing we want to do is split our dataset into training and test samples. The reason why we do this, is so that we can simulate a real life situation by generating predictions for our test sample, without showing the predictive model these data points. This gives us the ability to see how well our model is able to generalise to new data, which is critical.

A typical % to dedicate to testing is between 20-30, for this example we will use a 75-25% split between train and test respectively.

In [15]:
# Make a copy of our data
train_df = df.copy()

# Separate target variable from independent variables
y = df['churn']
X = df.drop(columns=['id', 'churn'])
print(X.shape)
print(y.shape)

(14606, 61)
(14606,)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(10954, 61)
(10954,)
(3652, 61)
(3652,)


### Model training

Once again, we are using a `Random Forest` classifier in this example. A Random Forest sits within the category of `ensemble` algorithms because internally the `Forest` refers to a collection of `Decision Trees` which are tree-based learning algorithms. As the data scientist, you can control how large the forest is (that is, how many decision trees you want to include).

The reason why an `ensemble` algorithm is powerful is because of the laws of averaging, weak learners and the central limit theorem. If we take a single decision tree and give it a sample of data and some parameters, it will learn patterns from the data. It may be overfit or it may be underfit, but that is now our only hope, that single algorithm. 

With `ensemble` methods, instead of banking on 1 single trained model, we can train 1000's of decision trees, all using different splits of the data and learning different patterns. It would be like asking 1000 people to all learn how to code. You would end up with 1000 people with different answers, methods and styles! The weak learner notion applies here too, it has been found that if you train your learners not to overfit, but to learn weak patterns within the data and you have a lot of these weak learners, together they come together to form a highly predictive pool of knowledge! This is a real life application of many brains are better than 1.

Now instead of relying on 1 single decision tree for prediction, the random forest puts it to the overall views of the entire collection of decision trees. Some ensemble algorithms using a voting approach to decide which prediction is best, others using averaging. 

As we increase the number of learners, the idea is that the random forest's performance should converge to its best possible solution.

Some additional advantages of the random forest classifier include:

- The random forest uses a rule-based approach instead of a distance calculation and so features do not need to be scaled
- It is able to handle non-linear parameters better than linear based models

On the flip side, some disadvantages of the random forest classifier include:

- The computational power needed to train a random forest on a large dataset is high, since we need to build a whole ensemble of estimators.
- Training time can be longer due to the increased complexity and size of thee ensemble

In [17]:
# Add model training in here!
model = RandomForestClassifier(n_estimators=100,max_depth=10) # Add parameters to the model!
model.fit(X,y) # Complete this method call!

RandomForestClassifier(max_depth=10)

### Evaluation

Now let's evaluate how well this trained model is able to predict the values of the test dataset.

In [18]:
# Generate predictions here!
predictions = model.predict(X_test)
tn, fp, fn, tp = metrics.confusion_matrix(y_test, predictions).ravel()

In [19]:
y_test.value_counts()

churn
0    3286
1     366
Name: count, dtype: int64

In [20]:
print(f"True positives: {tp}")
print(f"False positives: {fp}")
print(f"True negatives: {tn}")
print(f"False negatives: {fn}\n")

print(f"Accuracy: {metrics.accuracy_score(y_test, predictions)}")
print(f"Precision: {metrics.precision_score(y_test, predictions)}")
print(f"Recall: {metrics.recall_score(y_test, predictions)}")

True positives: 16
False positives: 0
True negatives: 3286
False negatives: 350

Accuracy: 0.9041621029572837
Precision: 1.0
Recall: 0.04371584699453552


In [22]:
# Calculate performance metrics here!
feature_importances = pd.DataFrame({
    'features': X_train.columns,
    'importance': model.feature_importances_
}).sort_values(by='importance', ascending=True).reset_index()

In [23]:
plt.figure(figsize=(15, 25))
plt.title('Feature Importances')
plt.barh(range(len(feature_importances)), feature_importances['importance'], color='b', align='center')
plt.yticks(range(len(feature_importances)), feature_importances['features'])
plt.xlabel('Importance')
plt.show()

In [24]:
proba_predictions = model.predict_proba(X_test)
probabilities = proba_predictions[:, 1]

In [25]:
X_test = X_test.reset_index()
X_test.drop(columns='index', inplace=True)
     

In [26]:
X_test['churn'] = predictions.tolist()
X_test['churn_probability'] = probabilities.tolist()
X_test.to_csv('out_of_sample_data_with_predictions.csv')

BUSINESS IMPACT OF DISCOUNT
LOADING THE DATA WE HAVE PREPARED ABOVE


In [27]:
test_df = pd.read_csv('out_of_sample_data_with_predictions.csv')
test_df.drop(test_df.columns[0], axis=1, inplace=True)
test_df.head()

cons_12m  cons_gas_12m  cons_last_month  forecast_cons_12m  \
0  4.173390      4.193598         3.165541           3.168368   
1  4.075036      0.000000         3.003461           3.305998   
2  3.848435      0.000000         0.000000           2.756568   
3  4.345433      0.000000         0.000000           3.516690   
4  4.185655      0.000000         3.355834           3.208148   

   forecast_discount_energy  forecast_meter_rent_12m  \
0                       0.0                 2.073828   
1                      30.0                 1.255031   
2                       0.0                 1.289366   
3                       0.0                 1.304921   
4                       0.0                 2.123067   

   forecast_price_energy_off_peak  forecast_price_energy_peak  \
0                        0.114667                    0.099548   
1                        0.198096                    0.000000   
2                        0.141434                    0.000000   
3                        0.142996                    0.000000   
4                        0.120372                    0.103487   

   forecast_price_pow_off_peak  has_gas  ...  channel_MISSING  \
0                    40.606701        1  ...                0   
1                    46.305378        0  ...                0   
2                    44.311378        0  ...                0   
3                    44.311378        0  ...                0   
4                    40.606701        0  ...                0   

   channel_ewpakwlliwisiwduibdlfmalxowmwpci  \
0                                         0   
1                                         0   
2                                         0   
3                                         0   
4                                         0   

   channel_foosdfpfkusacimwkcsosbicdxkicaua  \
0                                         1   
1                                         1   
2                                         1   
3                                         1   
4                                         1   

   channel_lmkebamcaaclubfxadlmueccxoimlema  \
0                                         0   
1                                         0   
2                                         0   
3                                         0   
4                                         0   

   channel_usilxuppasemubllopkaafesmlibmsdf  \
0                                         0   
1                                         0   
2                                         0   
3                                         0   
4                                         0   

   origin_up_kamkkxfxxuwbdslkwifmmcsiusiuosws  \
0                                           0   
1                                           0   
2                                           0   
3                                           0   
4                                           0   

   origin_up_ldkssxwpmemidmecebumciepifcamkci  \
0                                           0   
1                                           0   
2                                           0   
3                                           0   
4                                           1   

   origin_up_lxidpiddsbxsbosboudacockeimpuepw  churn  churn_probability  
0                                           1      0           0.066826  
1                                           1      0           0.144403  
2                                           1      0           0.056734  
3                                           1      0           0.095808  
4                                           0      0           0.083137  

[5 rows x 63 columns]

In [28]:
# Electricity revenue for each customer consists of energy consumption (amount * price) and the meter rent
test_df['basecase_revenue'] = (np.power(10, test_df['forecast_cons_12m']) + 1) * test_df['forecast_price_energy_off_peak'] + test_df['forecast_meter_rent_12m']

# Taking churn into account
test_df['basecase_revenue_after_churn'] = test_df['basecase_revenue'] * (1 - 0.919 * test_df['churn'])

test_df.head()

cons_12m  cons_gas_12m  cons_last_month  forecast_cons_12m  \
0  4.173390      4.193598         3.165541           3.168368   
1  4.075036      0.000000         3.003461           3.305998   
2  3.848435      0.000000         0.000000           2.756568   
3  4.345433      0.000000         0.000000           3.516690   
4  4.185655      0.000000         3.355834           3.208148   

   forecast_discount_energy  forecast_meter_rent_12m  \
0                       0.0                 2.073828   
1                      30.0                 1.255031   
2                       0.0                 1.289366   
3                       0.0                 1.304921   
4                       0.0                 2.123067   

   forecast_price_energy_off_peak  forecast_price_energy_peak  \
0                        0.114667                    0.099548   
1                        0.198096                    0.000000   
2                        0.141434                    0.000000   
3                        0.142996                    0.000000   
4                        0.120372                    0.103487   

   forecast_price_pow_off_peak  has_gas  ...  \
0                    40.606701        1  ...   
1                    46.305378        0  ...   
2                    44.311378        0  ...   
3                    44.311378        0  ...   
4                    40.606701        0  ...   

   channel_foosdfpfkusacimwkcsosbicdxkicaua  \
0                                         1   
1                                         1   
2                                         1   
3                                         1   
4                                         1   

   channel_lmkebamcaaclubfxadlmueccxoimlema  \
0                                         0   
1                                         0   
2                                         0   
3                                         0   
4                                         0   

   channel_usilxuppasemubllopkaafesmlibmsdf  \
0                                         0   
1                                         0   
2                                         0   
3                                         0   
4                                         0   

   origin_up_kamkkxfxxuwbdslkwifmmcsiusiuosws  \
0                                           0   
1                                           0   
2                                           0   
3                                           0   
4                                           0   

   origin_up_ldkssxwpmemidmecebumciepifcamkci  \
0                                           0   
1                                           0   
2                                           0   
3                                           0   
4                                           1   

   origin_up_lxidpiddsbxsbosboudacockeimpuepw  churn  churn_probability  \
0                                           1      0           0.066826   
1                                           1      0           0.144403   
2                                           1      0           0.056734   
3                                           1      0           0.095808   
4                                           0      0           0.083137   

   basecase_revenue  basecase_revenue_after_churn  
0        171.157200                    171.157200  
1        402.203316                    402.203316  
2         82.176885                     82.176885  
3        471.357082                    471.357082  
4        196.633386                    196.633386  

[5 rows x 65 columns]

Calculate the estimated benefits and costs of intervention
Under the assumptions provided, we can consider the impact of offering discounts to customers on revenue retention and potential loss due to false positives. Let's analyze these effects:

True Positives: Customers who would have churned but are retained due to the discount. In this scenario, the revenue is retained, resulting in revenue retention compared to the no-discount scenario. The retained revenue is calculated as 0.8 times the base case revenue (1 - discount_fraction).

False Positives: Customers who are offered a discount but would not have churned otherwise. Giving these customers a discount results in reduced revenue since they would have stayed without the discount. The loss in revenue is equal to the discount applied to these customers.

It's important to note that the assumption of all customers accepting the discount might not hold true in reality. Additionally, the accuracy of identifying true positives and false positives is dependent on the quality of the model and the observed dependent variable.

Overall, offering discounts can have a trade-off between revenue retention for true positives and potential revenue loss for false positives. Balancing these effects and optimizing the discount strategy is crucial to maximize revenue and minimize unnecessary discount

In [29]:
def get_rev_delta(pred: pd.DataFrame, cutoff: float=0.5, discount: float=0.2) -> float:
    """
    Get the delta of revenues for offering discount for all customers with predicted churn risk >= cutoff
    """
    pred['discount_revenue'] = pred['basecase_revenue_after_churn']
    # If a customer is predicted to churn, giving them a discount can keep them for the full year regardless of whether the prediction was correct or not.
    pred.loc[pred['churn_probability'] >= cutoff, 'discount_revenue'] = pred['basecase_revenue'] * (1 - discount)
    # Save the revenue delta for each customer in a separate column
    pred['revenue_delta'] = pred['discount_revenue'] - pred['basecase_revenue_after_churn']
    return pred['revenue_delta'].sum()
# Generate a list of possible cutoffs and the corresponding overall revenue deltas
rev_deltas = pd.Series({cutoff: get_rev_delta(test_df, cutoff=cutoff) for cutoff in np.arange(0, 1, 0.01)})

def plot_tradeoff(rev_deltas: pd.Series):
    # Plot the revenue deltas
    rev_deltas.plot()
    # Mark optimal point
    max_pred = rev_deltas.idxmax()
    plt.scatter(max_pred, rev_deltas.loc[max_pred], s=100, c='orange')
    # Reference line for break-even
    plt.hlines(0, 0, 1)
    plt.show()
    print(f'Maximum benefit at cutoff {max_pred} with revenue delta of ${rev_deltas.loc[max_pred]:,.2f}')

plot_tradeoff(rev_deltas)

Maximum benefit at cutoff 0.5 with revenue delta of $7,766.71
